This is an implementation of depolying CNN gensture reconition model to edge device (e.g. Sony Spresense)

Step Overview:
1. Conver Pytorch Model to Onnx Model
2. Conver Onnx Model to Keras Model
3. Conver Keras Model to quantization aware model
4. Retraining quantization aware model
5. Convert it to Tensorflow Lite Model
6. Convert the model to hex header file


## Create Representive Data

In [ ]:
import numpy as np
import scipy.io
import torch
import json
import os
config = json.load(open('config.json', 'r'))
# Load input data from a MATLAB file
for i in range(1,9):
    input_data = scipy.io.loadmat(os.path.join(config["data_path"],f"001-00{i}-005.mat"))['Data']

    # Convert the input data to a PyTorch tensor
    input_data = torch.asarray(input_data)

    # Reshape the input data to match the expected shape of the model
    input_data = input_data.reshape(-1, 1, 8, 24)

    # Normalize the input data by subtracting the mean and dividing by the standard deviation
    input_data = (input_data - input_data.mean()) / input_data.std()

    input_data = input_data.cpu().detach().numpy()

    np.save(f"representive_data_{i-1}",input_data)

# Conver Pytorch Model to Onnx Model

In [1]:
import torch
import torchvision
from models.mobilenetv1 import MobilenetV1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dummy_input = torch.randn(1, 1, 8, 24, device=device)
model = MobilenetV1(ch_in=1, n_classes=8, Global_ratio=0.4).to(device)
model.load_state_dict(torch.load(
    "pretrain_model/MobilenetV1_Param@529.28 k_MAC@903.38 KMac_Acc@96.629.pt"))
model.eval()

# print(model)
torch.onnx.export(model, dummy_input,
                  "pretrain_model/onnx_model/MobilenetV1.onnx", verbose=True)


/home/zhenyulin/anaconda3/envs/sEMG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


graph(%input.1 : Float(1, 1, 8, 24, strides=[192, 192, 24, 1], requires_grad=0, device=cuda:0),
      %fc.weight : Float(8, 409, strides=[409, 1], requires_grad=1, device=cuda:0),
      %fc.bias : Float(8, strides=[1], requires_grad=1, device=cuda:0),
      %251 : Float(12, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cuda:0),
      %252 : Float(12, strides=[1], requires_grad=0, device=cuda:0),
      %254 : Float(12, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cuda:0),
      %255 : Float(12, strides=[1], requires_grad=0, device=cuda:0),
      %257 : Float(25, 12, 1, 1, strides=[12, 1, 1, 1], requires_grad=0, device=cuda:0),
      %258 : Float(25, strides=[1], requires_grad=0, device=cuda:0),
      %260 : Float(25, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cuda:0),
      %261 : Float(25, strides=[1], requires_grad=0, device=cuda:0),
      %263 : Float(51, 25, 1, 1, strides=[25, 1, 1, 1], requires_grad=0, device=cuda:0),
      %264 : Float(51, strides=[1

# Conver Onnx Model to Keras Model

In [ ]:
# Install library
%cd onnx2keras
!pip install -e .
%cd ..

In [2]:
import tensorflow as tf
import onnx

onnx_model = onnx.load("pretrain_model/onnx_model/MobilenetV1.onnx")
from onnx2keras import onnx_to_keras
model = onnx_to_keras(onnx_model, ['input.1'],name_policy='renumerate',verbose=False,change_ordering=True)
model.summary()

2023-09-11 03:22:42.011475: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 03:22:42.044193: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 03:22:42.567943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/zhenyulin/anaconda3/envs/sEMG/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input.1 (InputLayer)        [(None, 8, 24, 1)]        0         
                                                                 
 LAYER_0_pad (ZeroPadding2D)  (None, 10, 26, 1)        0         
                                                                 
 LAYER_0 (Conv2D)            (None, 4, 12, 12)         120       
                                                                 
 LAYER_1 (Activation)        (None, 4, 12, 12)         0         
                                                                 
 LAYER_2_pad (ZeroPadding2D)  (None, 6, 14, 12)        0         
                                                                 
 LAYER_2 (DepthwiseConv2D)   (None, 4, 12, 12)         120       
                                                                 
 LAYER_3 (Activation)        (None, 4, 12, 12)         0     

## Quantization aware training

In [3]:
import tensorflow_model_optimization as tfmot
quantize_model = tfmot.quantization.keras.quantize_model
q_aware_model = quantize_model(model)
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

from utils.ICE_lab_data_preprocessing import ICE_lab_data_preprocessing as utils

data,label,num_classes = utils().extra_data("data/Training_Trimmed")
from sklearn.model_selection import train_test_split

training_data, testing_data, training_label, testing_label = train_test_split(data, label, test_size=0.33, random_state=42)
train_data = tf.data.Dataset.from_tensor_slices((training_data, training_label))
test_data = tf.data.Dataset.from_tensor_slices((testing_data, testing_label))

training_data = training_data.reshape(-1,8,24,1)
testing_data = testing_data.reshape(-1,8,24,1)
training_data = utils().NormalizeData(training_data)
testing_data = utils().NormalizeData(testing_data)
q_aware_model.fit(training_data,training_label,
                  batch_size=1000, epochs=15)
_, q_aware_model_accuracy = q_aware_model.evaluate(
   testing_data, testing_label, batch_size=1000,verbose=True)
print('Quant test accuracy:', q_aware_model_accuracy)




/home/zhenyulin/anaconda3/envs/sEMG/lib/python3.9/site-packages/tensorflow_model_optimization/python/core/quantization/keras/quantize.py:216: UserWarning: Lambda layers are not supported by automatic model annotation because the internal functionality cannot always be determined by serialization alone. We recommend that you make a custom layer and add a custom QuantizeConfig for it instead. This layer will not be quantized which may lead to unexpected results.
  warnings.warn(
Processing Files: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]


Epoch 1/15
659/659 [==============================] - 50s 71ms/step - loss: 0.2220 - accuracy: 0.9226
Epoch 2/15
659/659 [==============================] - 47s 71ms/step - loss: 0.1282 - accuracy: 0.9547
Epoch 3/15
361/659 [===============>..............] - ETA: 20s - loss: 0.1086 - accuracy: 0.9622

KeyboardInterrupt: 

In [4]:
q_aware_model.save("pretrain_model/q_ware_model")

2023-09-11 03:25:08.461761: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,409,1,1]
	 [[{{node inputs}}]]
2023-09-11 03:25:08.464945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,409,1,1]
	 [[{{node inputs}}]]
2023-09-11 03:25:11.119097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,409,1,1]
	 [[{{node inputs}}]]
2023

INFO:tensorflow:Assets written to: pretrain_model/q_ware_model/assets


INFO:tensorflow:Assets written to: pretrain_model/q_ware_model/assets


# Convert it to Tensorflow Lite Model

In [5]:
import numpy as np
import os
def representative_dataset():
    data = np.load("representive_data_0.npy")
    for i in range(1):
        temp_data = data[i]
        temp_data = temp_data.reshape(1,8,24,1)
        yield [temp_data.astype(np.float32)]

import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("pretrain_model/q_ware_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()
tflite_model_size = len(tflite_model) / 1024
print('Quantized model size = %dKBs.' % tflite_model_size)
# Save the model
with open("pretrain_model/tf_lite_model/mobilenetv1.tflite", 'wb') as f:
    f.write(tflite_model)

2023-09-11 03:25:20.005171: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-11 03:25:20.005199: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-11 03:25:20.005487: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: pretrain_model/q_ware_model
2023-09-11 03:25:20.017761: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-09-11 03:25:20.017775: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: pretrain_model/q_ware_model
2023-09-11 03:25:20.046806: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-09-11 03:25:20.057107: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-09-11 03:25:20.371894: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: pr

# Simulate Model Accuracy

In [6]:
import numpy as np
import tensorflow as tf
import os

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="pretrain_model/tf_lite_model/mobilenetv1.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)
for j in range(8):
    ori_input_data = np.load(f"representive_data_{j}.npy")
    ori_input_data = ori_input_data.astype(np.float32)
    # ori_input_data = ori_input_data.reshape(-1,8,24,1)
    correct = 0
    print("Total Sample Size:",ori_input_data.shape[0])
    for i in range(ori_input_data.shape[0]):
        input_data = np.expand_dims(ori_input_data[i], 0)
        input_data = input_data.reshape(-1,8,24,1)
        interpreter.set_tensor(input_details[0]['index'], input_data)

        interpreter.invoke()

        # get_tensor() returns a copy of the tensor data
        # use tensor() in order to get a pointer to the tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        if np.argmax(output_data) == j:
            correct += 1
    print("Prediction Correct Size:",correct) #Total:30720
    print("Accuracy",round(correct/int(ori_input_data.shape[0]),2))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Total Sample Size: 30720
Prediction Correct Size: 27396
Accuracy 0.89
Total Sample Size: 30720
Prediction Correct Size: 26408
Accuracy 0.86
Total Sample Size: 30720
Prediction Correct Size: 28503
Accuracy 0.93
Total Sample Size: 30720
Prediction Correct Size: 26627
Accuracy 0.87
Total Sample Size: 30720
Prediction Correct Size: 29094
Accuracy 0.95
Total Sample Size: 30720
Prediction Correct Size: 574
Accuracy 0.02
Total Sample Size: 30720
Prediction Correct Size: 24320
Accuracy 0.79
Total Sample Size: 30880
Prediction Correct Size: 3436
Accuracy 0.11


# Convert TFlite file to hex header file

In [ ]:
def convert_to_c_array(bytes) -> str:
  hexstr = binascii.hexlify(bytes).decode("UTF-8")
  hexstr = hexstr.upper()
  array = ["0x" + hexstr[i:i + 2] for i in range(0, len(hexstr), 2)]
  array = [array[i:i+10] for i in range(0, len(array), 10)]
  return ",\n  ".join([", ".join(e) for e in array])

tflite_binary = open('pretrain_model/tf_lite_model/mobilenetv1.tflite', 'rb').read()
ascii_bytes = convert_to_c_array(tflite_binary)
header_file = "const unsigned char model_tflite[] = {\n  " + ascii_bytes + "\n};\nunsigned int model_tflite_len = " + str(len(tflite_binary)) + ";"
with open("pretrain_model/tf_lite_model/model.h", "w") as f:
    f.write(header_file)